# CSE547 - Colab 4
## Collaborative Filtering

Adapted from Stanford's CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 5.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=474a6ade3320ac1b7ec465627ac806de34a0d1c83edb52363ff4a643e6ce3ecd
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [ ]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [ ]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [ ]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [ ]:
# YOUR CODE HERE
# YOUR CODE HERE
type(training)
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql import Row

#model = als.fit(training)
           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Using the training set, train models with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

Model 1: 

*   maximal iteration = 10
*   **rank = 10**

*   regularization = 0.1
*   Drop rows with a NaN value in rating



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

als = ALS(maxIter=10, regParam=0.1,rank = 10, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)
type(als)



pyspark.ml.recommendation.ALS

In [ ]:
# YOUR CODE HERE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 

Now compute the RMSE on the test dataset.


In [ ]:
# YOUR CODE HERE
test_predictions = model.transform(test)
RMSE = evaluator.evaluate(test_predictions)

In [ ]:
RMSE

0.9345635449533132

To experiment with different values of rank, repeat the process of computing RMSE for the following models (Feel free to just edit the code you wrote above):

Model 2: 

*   Model 1 with **rank = 5**

Model 3: 

*   Model 1 with **rank = 100**

In [ ]:
# YOUR CODE 
als = ALS(maxIter=10, regParam=0.1,rank = 5, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model1 = als.fit(training)
type(als)
# YOUR CODE HERE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
# YOUR CODE HERE
test_predictions1 = model1.transform(test)
RMSE1 = evaluator.evaluate(test_predictions1)

In [ ]:
RMSE1

0.9310683356424587

In [ ]:
# YOUR CODE 
als = ALS(maxIter=10, regParam=0.1,rank = 100, userCol="user_id", itemCol="item_id", ratingCol="rating",
          coldStartStrategy="drop")
model1 = als.fit(training)
type(als)
# YOUR CODE HERE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
# YOUR CODE HERE
test_predictions1 = model1.transform(test)
RMSE1 = evaluator.evaluate(test_predictions1)

In [ ]:
RMSE1

0.9284554255448575

Now experiment with different regularization values using Model 3 (**rank = 100**). Which regularization gives the lowest RMSE?  (Feel free to just edit the code you wrote above)

*   1
* 0.3
* 0.1
* 0.03
* 0.01



In [ ]:
# YOUR CODE HERE


At this point, you can use the trained model to produce the top-K recommendations for each user. 

Using model 3 (**rank 100**, **regularization 0.1**) find the most frequent #1 movie recommendation across all users. You may find it easier to convert the recommendation output to a Pandas Dataframe. Which movie is it?

Hint: One approach is described on GradeScope.

In [ ]:
# YOUR CODE HERE
# Generate top 5 movie recommendations for each user
userRecs = model1.recommendForAllUsers(10)
# Generate top 5 user recommendations for each movie
movieRecs = model1.recommendForAllItems(10)

In [ ]:
movieRecs.limit(10).show()

+-------+--------------------+
|item_id|     recommendations|
+-------+--------------------+
|      1|[{688, 5.0581913}...|
|      3|[{366, 4.382543},...|
|      5|[{907, 4.657202},...|
|      6|[{662, 4.5710464}...|
|      9|[{98, 4.946497}, ...|
|     12|[{357, 5.3115435}...|
|     13|[{52, 4.4154615},...|
|     15|[{939, 5.030822},...|
|     16|[{366, 4.6572943}...|
|     17|[{366, 4.8047223}...|
+-------+--------------------+



In [ ]:
nrecommendations = userRecs\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('user_Id', col("rec_exp.item_id"), col("rec_exp.rating"))

In [ ]:
nrecommendations1 = Window.partitionBy("user_Id").orderBy(col("rating").desc())
df2=nrecommendations.withColumn("row",row_number().over(nrecommendations1))

In [ ]:
df3=df2.filter(col("row") <= 1)

In [ ]:
df3.show()

+-------+-------+---------+---+
|user_Id|item_id|   rating|row|
+-------+-------+---------+---+
|      1|    119| 4.835057|  1|
|      3|    320| 4.720159|  1|
|      5|    390| 4.511979|  1|
|      6|    493|4.4892645|  1|
|      9|   1643| 4.972594|  1|
|     12|    318|4.8855743|  1|
|     13|    867| 4.838311|  1|
|     15|    936| 4.322406|  1|
|     16|     64| 5.405167|  1|
|     17|    150|4.3070064|  1|
|     19|      8| 4.504151|  1|
|     20|     22|4.1078153|  1|
|     22|     50|4.7993636|  1|
|     26|    313|4.0866733|  1|
|     27|   1449| 4.118521|  1|
|     28|     50| 4.564588|  1|
|     31|   1643|4.7312665|  1|
|     34|   1449| 4.865592|  1|
|     35|    300|4.2576175|  1|
|     37|     50| 4.370982|  1|
+-------+-------+---------+---+
only showing top 20 rows



In [ ]:
Day_count = df3.groupby('item_id').count()
Day_count.orderBy(desc('count')).take(1)

[Row(item_id=1449, count=138)]

In [ ]:
items.collect()[1448]

Row(item_id=1449, movie='Pather Panchali (1955)')

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!